<a href="https://colab.research.google.com/github/cincinnatilibrary/collection-analysis/blob/master/reports/leased_items_checkouts_by_branch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CHPL - Collection Analysis - **Leased Items**

<img src="https://ilsweb.cincinnatilibrary.org/img/CHPL_Brandmark_Primary.png" alt="CHPL" title="CHPL" width="300"/>

Data sources / Documentation:

* https://ilsweb.cincinnatilibrary.org/collection-analysis/

In [1]:
#@title
import pandas as pd
import numpy as np
import altair as alt
# if we want to save output as .png use this
# NOTE: use !pip install -U altair_saver above to download / install first
# from altair_saver import save
from urllib.parse import urlencode
import requests

base_url = 'https://collection-analysis.cincy.pl/current_collection'

In [2]:
#@title
sql = """\
-- find the ciculation of leased items by stat group of branch
select
  strftime('%Y-%m', transaction_day) as circ_month,
  stat_group_branch_name,
  -- bib_record_num,
  count(*) as count_checkouts
from
  circ_leased_items
where
  op_code = 'o'
group by
  1,
  2
order by
  1,
  2
"""

df = pd.read_json(
    requests.get(
      url=base_url + '.json',
      params={
        'sql': sql,
        '_shape': 'array'
      }
    ).content
)

# df.head()

## Find Aggregated Check-out Circulations For Leased Items by Branch and Month 

(previous 180 days)

---



In [3]:
#@title
# color="#0090bd"
chart = alt.Chart(df).mark_bar().encode(
    # x='end_date:T',
    x=alt.X(
        'circ_month',
        axis=alt.Axis(title='circulation month', labelAngle=-70),
        # sort='-y'
    ),
    y=alt.Y(
        'count_checkouts:Q',
        title = 'count total leased item checkouts',
        # scale = alt.Scale(domain=[0,100]),
    ),
    color=alt.Color(
        'stat_group_branch_name',
        legend=None
    ),
    tooltip=['stat_group_branch_name', 'circ_month', 'count_checkouts']
).properties(
    title="",
    # .format(df.iloc[0]['branch_name']),
    width=250,
).facet(
    facet='stat_group_branch_name',
    columns=6
)

chart.display()

alt.FacetChart(...)

In [4]:
#@title
sql = """\
select
  l.bib_record_num,
  -- l.itype_code_num,
  p.itype_name as item_format,
  cast(l.bib_record_num as TEXT) || ' -- ' || case
    when length(bib.best_title) > 30 then substr(bib.best_title, 1, 30) || ' ... '
    else bib.best_title
  end || coalesce(
    ' / ' || nullif(
      case
        when length(bib.best_author) > 20 then substr(bib.best_author, 1, 20) || ' ... '
        else bib.best_author
      end,
      ''
    ),
    ''
  ) || coalesce(' (' || bib.publish_year || ')', '') as title,
  strftime('%Y-%m', transaction_day) as circ_month,
  count(*) as count_checkouts
from
  circ_leased_items as l
  join bib on bib.bib_record_num = l.bib_record_num
  join itype_property as p on p.itype_code = l.itype_code_num
where
  op_code = 'o'
group by
  1,
  2,
  3,
  4
order by
  1 DESC,
  2 ASC
"""

df = pd.read_json(
    requests.get(
      url=base_url + '.json',
      params={
        'sql': sql,
        '_shape': 'array'
      }
    ).content
)

# df.head()

## Find Aggregated Check-out Circulations For Leased Items by Title and Month 

(previous 180 days)

**Note**: if this is difficult to view in the pane below, expand the file menu ( image.png icon to the left) 

<--

and click to download the `output.html` file



---

In [5]:
#@title
# color="#0090bd"
chart = alt.Chart(df).mark_bar().encode(
    # x='end_date:T',
    x=alt.X(
        'circ_month',
        axis=alt.Axis(title='circulation month', labelAngle=-70),
        # sort=manual_ordered
    ),
    y=alt.Y(
        'count_checkouts:Q',
        title = 'count total leased item checkouts',
        # scale = alt.Scale(domain=[0,100]),
    ),
    color=alt.Color(
        'item_format',
        # legend=None
    ),
    tooltip=['item_format', 'bib_record_num', 'title', 'circ_month', 'count_checkouts']
).properties(
    title="",
    # .format(df.iloc[0]['branch_name']),
    width=400,
).facet(
    facet='title',
    columns=3
)

chart.save('output.html')

from IPython.display import HTML
HTML('output.html')